In [8]:
import pandas as pd


In [9]:
data_train=pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\train.csv")
data_test=pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\test.csv")
sample_submission = pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\sample_submission.csv")

In [10]:
# Fill numeric NaNs with mean
data_train = data_train.fillna(data_train.mean(numeric_only=True))

# Fill categorical NaNs
data_train['installation_type'] = data_train['installation_type'].fillna('unknown')
data_train['error_code'] = data_train['error_code'].fillna('none')


In [11]:
# Same preprocessing for test set
data_test = data_test.fillna(data_test.mean(numeric_only=True))
data_test['installation_type'] = data_test['installation_type'].fillna('unknown')
data_test['error_code'] = data_test['error_code'].fillna('none')

# Save test ids for final submission
test_ids = data_test['id']


In [12]:
# Drop unnecessary columns for modeling
X_train = data_train.drop(columns=['id', 'efficiency'])
y_train = data_train['efficiency']

X_test = data_test.drop(columns=['id'])

# One-hot encode categorical variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Align columns (test set might be missing some dummy columns)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)


In [ ]:
# Predict on test set
preds = model.predict(X_test)

# Predict on training set for evaluation
train_preds = model.predict(X_train)

# R² Score on training data
from sklearn.metrics import r2_score, mean_squared_log_error
import numpy as np

r2 = r2_score(y_train, train_preds)

# Clip predictions to non-negative for RMSLE
train_preds_clipped = np.maximum(train_preds, 0)
y_train_clipped = np.maximum(y_train, 0)

rmsle = np.sqrt(mean_squared_log_error(y_train_clipped, train_preds_clipped))

print(f"Training R² Score: {r2:.4f}")
print(f"Training RMSLE: {rmsle:.4f}")

# Prepare submission file
submission = pd.DataFrame({
    'id': test_ids,
    'efficiency': preds
})

# Ensure correct column order
submission = submission[sample_submission.columns]

# Save submission to CSV
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved successfully.")


In [2]:
import pandas as pd

In [3]:
data_train=pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\train.csv")
data_test=pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\test.csv")
sample_submission = pd.read_csv(r"C:\Users\HP\Desktop\hackathon\zelextra  x aws\dataset\sample_submission.csv")

In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import r2_score, mean_squared_log_error



# --- Preprocessing ---
# Fill numeric NaNs
data_train = data_train.fillna(data_train.mean(numeric_only=True))
data_test = data_test.fillna(data_test.mean(numeric_only=True))

# Fill categorical NaNs
for col in ['installation_type', 'error_code']:
    data_train[col] = data_train[col].fillna('unknown')
    data_test[col] = data_test[col].fillna('unknown')

# --- Target and Features ---
X_train = data_train.drop(columns=['id', 'efficiency'])
y_train = data_train['efficiency']
X_test = data_test.drop(columns=['id'])
test_ids = data_test['id']

# --- Identify categorical columns (by name or index) ---
cat_features = [
    'humidity',
    'wind_speed',
    'pressure',
    'string_id',
    'error_code',
    'installation_type'
]


# --- CatBoost Pool ---
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, cat_features=cat_features)

# --- Train CatBoost model ---
model = CatBoostRegressor(verbose=100, random_seed=42)
model.fit(train_pool)

# --- Evaluate on training data ---
train_preds = model.predict(X_train)
r2 = r2_score(y_train, train_preds)
rmsle = np.sqrt(mean_squared_log_error(np.maximum(y_train, 0), np.maximum(train_preds, 0)))

print(f"Training R² Score: {r2:.4f}")
print(f"Training RMSLE: {rmsle:.4f}")

# --- Predict on test data ---
preds = model.predict(test_pool)

# --- Submission ---
submission = pd.DataFrame({
    'id': test_ids,
    'efficiency': preds
})

# Match submission format
submission = submission[sample_submission.columns]
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved successfully.")


Learning rate set to 0.065727
0:	learn: 0.1368357	total: 213ms	remaining: 3m 32s
100:	learn: 0.1023804	total: 6.68s	remaining: 59.4s
200:	learn: 0.1015606	total: 12.3s	remaining: 48.9s
300:	learn: 0.1007992	total: 18.5s	remaining: 43s
400:	learn: 0.1000153	total: 24.6s	remaining: 36.8s
500:	learn: 0.0991481	total: 30.7s	remaining: 30.6s
600:	learn: 0.0983579	total: 36.8s	remaining: 24.4s
700:	learn: 0.0974513	total: 42.8s	remaining: 18.3s
800:	learn: 0.0965321	total: 48.7s	remaining: 12.1s
900:	learn: 0.0955850	total: 54.5s	remaining: 5.99s
999:	learn: 0.0948668	total: 1m	remaining: 0us
Training R² Score: 0.5000
Training RMSLE: 0.0768
✅ submission.csv saved successfully.


In [5]:
print(X_train.dtypes)


temperature           float64
irradiance            float64
humidity               object
panel_age             float64
maintenance_count     float64
soiling_ratio         float64
voltage               float64
current               float64
module_temperature    float64
cloud_coverage        float64
wind_speed             object
pressure               object
string_id              object
error_code             object
installation_type      object
dtype: object


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

# Copy data to avoid modifying original
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()

# Define categorical columns
cat_features = [
    'humidity',
    'wind_speed',
    'pressure',
    'string_id',
    'error_code',
    'installation_type'
]

# Apply Label Encoding for categorical features
le_dict = {}
for col in cat_features:
    le = LabelEncoder()
    # Combine train and test to avoid unseen labels
    X_combined = pd.concat([X_train_enc[col], X_test_enc[col]], axis=0)
    le.fit(X_combined.fillna("Unknown"))
    X_train_enc[col] = le.transform(X_train_enc[col].fillna("Unknown"))
    X_test_enc[col] = le.transform(X_test_enc[col].fillna("Unknown"))
    le_dict[col] = le  # Save encoders in case you want to inverse-transform later

# Initialize and train the XGBoost model
model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
model.fit(X_train_enc, y_train)

# Predict on test set
preds = model.predict(X_test_enc)

# Create submission
submission = pd.DataFrame({
    'id': test_ids,
    'efficiency': preds
})

submission = submission[sample_submission.columns]
submission.to_csv("submission.csv", index=False)


In [9]:
train_preds = model.predict(X_train_enc)

r2 = r2_score(y_train, train_preds)

print(f"Train R² Score: {r2:.4f}")
print(f"Train RMSLE: {rmsle:.4f}")


Train R² Score: 0.7464
Train RMSLE: 0.0768
